In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
from sentence_transformers import SentenceTransformer
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import sys
sys.path.append("/home/rtue/playground/jupyter-notebook/workspace/codes")
sys.path.append("/home/rtue/playground/python_projects/tool_keaml/src/utils")
sys.path.append("/home/rtue/playground/jupyter-notebook/keyword_extraction")
import util, evaluation_metrics
import encoding_pipeline
from training_data.data_prepare import DataPrepare
from youTube_pipeline_c import youtube_pipeline

import ast
from termcolor import colored
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import itertools

1.14.0


In [2]:
path_dw_id_kw='/home/rtue/playground/python_projects/resources/df_np_id_kw.csv'
path_dw_id_tit_abst_claim ='/home/rtue/playground/python_projects/resources/df_np_id_tit_abst_claim_detden.csv'

In [3]:
model = SentenceTransformer("AI-Growth-Lab/PatentSBERTa")
embedding_dim=768
max_len_token=model.get_max_seq_length()

DeepCTR version 0.9.0 detected. Your version is 0.8.5.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.9.0


In [4]:
data=DataPrepare()

data.set_der_abst()
data.set_full_text()

DataPrepare object has the following properties:
 df_full_text, dict_full_text, df_der_text, dict_der_text


In [5]:
data_id_kw, data_id_tit_abs_clm, data_kw=util.read_dataset(path_dw_id_kw, path_dw_id_tit_abst_claim)

rows_with_NaN: Empty DataFrame
Columns: [PPID, KEYWORD]
Index: []
rows_with_NaN: Empty DataFrame
Columns: [PPID, TIEN, ABEN, CLMEN, DETDEN]
Index: []
data_kw is merged version and len(data_kw): 271137


In [6]:
dict_occ_kw=util.get_occurance_dict(data_kw,['KEYWORD'])

occurance dictionary len: 79450


In [7]:
list_ele_1=util.get_list_ele_len_less_than(dict_occ_kw)
list_ele_2=util.get_list_ele_occ_less_than(dict_occ_kw, 100)
list_ele_3 = util.get_list_pha_len_less(dict_occ_kw)
list_ele_4 = util.get_list_ele_occ_more_than(dict_occ_kw)
lst_all=set(list_ele_1 + list_ele_2+list_ele_3+list_ele_4)

In [8]:
data_kw=util.remove_rows_contain_kw_in_list(lst_all, data_kw)
dict_occ_kw_clean=util.get_occurance_dict(data_kw,['KEYWORD'])

removing rows from the df
occurance dictionary len: 77


In [9]:
kw_list=list(dict_occ_kw_clean.keys())

In [10]:
f_name_test_dict='/home/rtue/playground/python_projects/resources/baseline_test_datasets/test_set_dict_baseline_embedding'
#/77_kws/train_test_sets/test_set_dict_10_de_20220222-134947'
test_set_dict=util.load_obj(f_name_test_dict)

In [76]:
count=0
for (k,v) in test_set_dict.items():
    print(count, k,v)
    count+=1

    len(test_set_dict)

0 US20100272700A1 {'ovarian cancer', 'crohn disease', 'culture medium', 'cervical cancer', 'effective amount', 'lung cancer', 'dendritic cell', 'rheumatoid arthritis', 'sjogren syndrome', 'pancreatic cancer', 'breast cancer', 'multiple myeloma', 'myasthenia gravis', 'prostate cancer', 'autoimmune disease'}
1 CN106478819A {'ovarian cancer', 'inflammatory disease', 'infectious disease', 'cervical cancer', 'monoclonal antibody', 'lung cancer', 'expression vector', 'therapeutic agent', 'pancreatic cancer', 'hodgkin lymphoma', 'colon cancer', 'breast cancer', 'respiratory syncytial virus', 'influenza virus', 'hepatitis virus', 'nucleotide sequence', 'prostate cancer'}
2 WO2004108899A2 {'crohn disease', 'rheumatoid arthritis', 'inflammatory bowel disease', 'sjogren syndrome', 'herpe simplex virus', 'systemic lupus erythematosus', 'ebola virus', 'lung cancer', 'bacterial infection', 'alzheimer disease', 'respiratory syncytial virus', 'dengue virus', 'breast cancer', 'myasthenia gravis', 'huma

In [11]:
candidate_embeddings = model.encode(kw_list)

In [12]:
candidate_embeddings.shape

(77, 768)

In [13]:
def assign_kw_based_on_embeddin(model, test_set, candidates, n):
    dict_result=dict()
    r = []
    p = []
    p_len = []
    hit = 0
    lst_preds=[]
    
    candidate_embeddings=model.encode(candidates)
    
    for (k,v) in test_set.items():
        #print(k,v)
        row=data.df_full_text.loc[data.df_full_text['PPID'] == k]
        #row=data_id_tit_abs_clm.loc[data_id_tit_abs_clm['PPID'] == k]
        tien=row['TIEN'].iloc[0]
        aben=row['ABEN'].iloc[0]
        clmen=row['CLMEN'].iloc[0]
        text=[tien+aben+clmen]
        
        
        row_np=data_id_tit_abs_clm.loc[data_id_tit_abs_clm['PPID'] == k]
        
        tien_np=row_np['TIEN'].iloc[0]
        aben_np=row_np['ABEN'].iloc[0]
        clmen_np=row_np['CLMEN'].iloc[0]
        detden_np=row_np['DETDEN'].iloc[0]
        
        set_full_kws=set(itertools.chain(util.literal_eval(tien_np), util.literal_eval(aben_np),util.literal_eval(clmen_np),util.literal_eval(detden_np)))

        if len(text)> max_len_token:
            doc_embedding = encoding_pipeline.get_vec_long_text(text, model)
        else:
            doc_embedding = model.encode(text)
    
        distances = cosine_similarity(doc_embedding, candidate_embeddings)
      
        
        keywords = [candidates[index] for index in distances.argsort()[0]]

        keywords = [kw for kw in keywords if kw in set_full_kws][:n]

        
        
        recall_score = evaluation_metrics.recall_at_N(v, keywords, n)
        precision= evaluation_metrics.precision_at_N( v, keywords)

        r.append(recall_score)

        if len(set(keywords))>0:
            p.append(precision)
            p_len.append(len(set(keywords)))

        #hit += len(set(set_gt_decoded) & set(pred_post))
    
    print("precision", np.mean(p))
    print("recall", np.mean(r))
    print('f_score:', evaluation_metrics.f_score(np.mean(p), np.mean(r)) )
    #p=[x for x in p if x>0]
    #print("precision after removing zeros", np.mean(p))
    #print('f1 after removing :', f_score(np.mean(p), np.mean(r)) )

In [14]:
n=10
assign_kw_based_on_embeddin(model, test_set_dict, kw_list, n)

precision 0.572
recall 0.4320704883981973
f_score: 0.492284799163932


In [15]:
n=15
assign_kw_based_on_embeddin(model, test_set_dict, kw_list, n)

precision 0.5642234432234432
recall 0.634145546421243
f_score: 0.5971445970288207


In [16]:
n=20
assign_kw_based_on_embeddin(model, test_set_dict, kw_list, n)

precision 0.5511065271587717
recall 0.770720197452473
f_score: 0.6426696079307608
